# 

## 1. Make sure you have the following packages in your environment installed

In [ ]:
#!pip install langchain==0.3.25 langchain-community==0.3.25 langchain-ollama==0.3.3 chromadb==1.0.12

## 2. Load your documents

In [40]:
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings, OllamaLLM
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

loader = DirectoryLoader(
    "./documents",
    glob="**/*.txt",
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"}
)
docs = loader.load()
print(f"One single document:\n'{docs[0].page_content}'\n\nFrom the file:\n'{docs[0].metadata['source']}'")

One single document:
'THE FOX AND THE CAT

It happened that the cat met the fox in a forest, and as she thought to herself: ‘He is clever and full of experience, and much esteemed in the world,’ she spoke to him in a friendly way. ‘Good day, dear Mr Fox, how are you? How is all with you? How are you getting on in these hard times?’ The fox, full of all kinds of arrogance, looked at the cat from head to foot, and for a long time did not know whether he would give any answer or not. At last he said: ‘Oh, you wretched beard-cleaner, you piebald fool, you hungry mouse-hunter, what can you be thinking of? Have you the cheek to ask how I am getting on? What have you learnt? How many arts do you understand?’ ‘I understand but one,’ replied the cat, modestly. ‘What art is that?’ asked the fox. ‘When the hounds are following me, I can spring into a tree and save myself.’ ‘Is that all?’ said the fox. ‘I am master of a hundred arts, and have into the bargain a sackful of cunning. You make me sorr

## 3. Chunk your documents

In [39]:
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
chunks = splitter.split_documents(docs)
print(f"One single document chunk:\n'{chunks[0].page_content}'\n\nFrom the file:\n'{chunks[0].metadata['source']}'")

One single document chunk:
'THE FOX AND THE CAT'

From the file:
'documents/the_fox_and_the_cat.txt'


## 4. Embed and index chunks in a vector database
Make sure to execute `ollama pull all-minilm` in the terminal before running the following snippet.

In [8]:
model = "all-minilm"            # 23M parameters
# model = "nomic-embed-text"    # 137M parameters
# model = "mxbai-embed-large"   # 334M parameters

embed_model = OllamaEmbeddings(model=model, base_url="http://127.0.0.1:11434")
vector_store = Chroma.from_documents(chunks, embed_model)
collection = vector_store._collection
print("Total embeddings:", collection.count())

Total embeddings: 63


In [10]:
all_data = collection.get(include=["documents", "embeddings"])
print(f"Documents: {all_data['documents'][1]}\n")
print(f"Embeddings: {all_data['embeddings'][1]}")

Documents: It happened that the cat met the fox in a forest, and as she thought to herself: ‘He is clever and full of experience, and much esteemed in the world,’ she spoke to him in a friendly way. ‘Good day, dear Mr Fox, how are you? How is all with you? How are you getting on in these hard times?’ The fox, full of all kinds of arrogance, looked at the cat from head to foot, and for a long time did not know whether he would give any answer or not. At last he said: ‘Oh, you wretched beard-cleaner, you piebald fool,

Embeddings: [ 4.92861606e-02  5.58096543e-02  1.91971660e-02  6.77148998e-02
  1.30003197e-02 -5.70437536e-02  3.91514152e-02 -8.34084768e-03
 -8.25796500e-02 -4.65176925e-02 -4.43423465e-02 -1.87217891e-02
 -2.63857730e-02  4.89936247e-02 -5.53207546e-02  2.54260339e-02
 -1.22084230e-01 -4.46500927e-02  2.82399636e-02  1.44945877e-02
 -3.57179157e-02  4.51493189e-02  9.16457325e-02  3.01709445e-03
 -3.62648889e-02 -5.26916273e-02 -5.23130894e-02 -4.16037962e-02
  1.248940

In [11]:
retriever = vector_store.as_retriever(search_kwargs={"k": 4})
print(retriever)

tags=['Chroma', 'OllamaEmbeddings'] vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x10e11ba10> search_kwargs={'k': 4}


## 5. Create a Retrieval-QA chain
Make sure to execute `ollama pull gemma3:1b` in the terminal before running the following snippet.

In [28]:
from langchain_ollama import OllamaLLM
from langchain.chains import RetrievalQA

# model = "gemma3:1b"     # 1B parameters
# model = "llama3.2"    # 3B parameters
model = "mistral"     # 7B parameters

llm = OllamaLLM(model=model, base_url="http://127.0.0.1:11434")
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",            # or "map_reduce", "refine", etc.
    return_source_documents=True,  # if you want the source chunks back
)
print(llm)

OllamaLLM
Params: {}


In [29]:
result = qa_chain.invoke({"query": "Who is Hans?"})
print(f"Answer: {result['result']}\n")
print(f"Source: {result['source_documents']}")

Answer:  Hans is a character from the story "Clever Hans." He has the ability to perform mathematical calculations and solve problems without any formal education. In this context, he appears as a mischievous character who manipulates Gretel and his mother for personal gain.

Source: [Document(metadata={'source': 'documents/clever_hans.txt'}, page_content='CLEVER HANS\n\nThe mother of Hans said: ‘Whither away, Hans?’ Hans answered: ‘To Gretel.’ ‘Behave well, Hans.’ ‘Oh, I’ll behave well. Goodbye, mother.’ ‘Goodbye, Hans.’ Hans comes to Gretel. ‘Good day, Gretel.’ ‘Good day, Hans. What do you bring that is good?’ ‘I bring nothing, I want to have something given me.’ Gretel presents Hans with a needle, Hans says: ‘Goodbye, Gretel.’ ‘Goodbye, Hans.’'), Document(metadata={'source': 'documents/clever_hans.txt'}, page_content='Hans takes Gretel, ties her to a rope, leads her to the rack, and binds her fast. Then Hans goes to his mother. ‘Good evening, mother.’ ‘Good evening, Hans. Where have

In [23]:
result['source_documents'][0].page_content

'CLEVER HANS\n\nThe mother of Hans said: ‘Whither away, Hans?’ Hans answered: ‘To Gretel.’ ‘Behave well, Hans.’ ‘Oh, I’ll behave well. Goodbye, mother.’ ‘Goodbye, Hans.’ Hans comes to Gretel. ‘Good day, Gretel.’ ‘Good day, Hans. What do you bring that is good?’ ‘I bring nothing, I want to have something given me.’ Gretel presents Hans with a needle, Hans says: ‘Goodbye, Gretel.’ ‘Goodbye, Hans.’'